# How to use LangChain and Azure OpenAI with Python


Langchain is an open source framework for developing applications using large language models (LLM). <br>

This guide will demonstrate how to setup and use Azure OpenAI models' API with LangChain.
    

## Set Up
The following libraries must be installed to use LangChain with Azure OpenAI.<br>

In [16]:
#%pip install --upgrade openai
#%pip install langchain

## API Configuation

After installing the necessary libraies, the API must be configured. The code below shows how to configure the API directly in your Python environment. 


In [25]:
from langchain.llms import AzureOpenAI 
import openai
import json
import os

# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

# Setting up the deployment name
deployment_name = config_details['DEPLOYMENT_NAME']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently Chat Completion API have the following versions available: 2023-07-01-preview
openai.api_version = config_details['OPENAI_API_VERSION']

## Deployed Model Setup

Azure OpenAI allows users to create and manage their own model deployments. To call the API, you must specify the deployment you want to use by passing in the deployment name and model name. 

In [33]:
# Create an instance of Azure OpenAI

# Replace the deployment name and model name with your own
llm = AzureOpenAI(
    deployment_name= deployment_name,
    model_name="text-davinci-002", 
)

# Run the LLM
llm("Write me a poem")

"\n\n\n\n\nThe world is a beautiful place,\nThe colors are so bright and true,\nAnd I feel so free and free,\nWhen I'm away from here.\n\nThe sky is so blue,\nAnd the sun is so warm,\nAnd I feel so free and free,\nWhen I'm away from here."

## PromptTemplates

Langchain provides a built in PromptsTemplate module to simplify the construction of prompts to get more specific answers.

In [40]:
from langchain import PromptTemplate

template = """
You are a skin care consulant that recommends products based on customer
needs and preferences.

What is a good {product_type} to help with {customer_request}?
"""

prompt = PromptTemplate(
input_variables=["product_type", "customer_request"],
template=template,
)

print(llm(
    prompt.format(
        product_type="face wash",
        customer_request = "acne prone skin"
    )
))


There are a number of good face washes that can help with acne prone skin. A few popular options include the Neutrogena Oil-Free Acne Wash, the Cetaphil Dermatological Gentle Skin Cleanser, and the La Roche-Posay Effaclar Medicated Gel Cleanser.


## Chains
There are many applications of chains that allow you to combine numerous LLM calls and actions.  <br>

### Simple Sequential Chains  <br>
Allow you to feed the output of one LLM Chain as input for another.

In [46]:
#from langchain.llms import AzureOpenAI
from langchain.chains import LLMChain
# from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

In [51]:
template = """Your job is to come up with a fun DIY project for the specified gender, age, and description of a kid.
% CHILD_DESCRIPTION
{child_description}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["child_description"], template=template)

# Holds my 'location' chain
description_chain = LLMChain(llm=llm, prompt=prompt_template)

In [55]:
template = """Given a DIY project, give a short and simple recipe step-by-step guide on how to complete the project and a materials list.
% DIY_PROJECT
{diy_project}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["diy_project"], template=template)

# Holds my 'meal' chain
diy_chain = LLMChain(llm=llm, prompt=prompt_template)

In [56]:
overall_chain = SimpleSequentialChain(chains=[description_chain, diy_chain], verbose=True)

In [57]:
review = overall_chain.run("5-year-old girl")



> Entering new SimpleSequentialChain chain...



A simple and fun DIY project for a 5-year-old girl is to make a paper doll. All you need is some paper, scissors, and crayons or markers. First, cut out a paper doll shape from the paper. Then, decorate the paper doll with clothes, hair, and facial features. You can also cut out paper furniture and accessories to create a paper doll scene.

1. Cut out a paper doll shape from the paper.
2. Decorate the paper doll with clothes, hair, and facial features.
3. Cut out paper furniture and accessories to create a paper doll scene.

Materials needed:

-Paper
-Scissors
-Crayons or markers

> Finished chain.
